# Cleaning / Preprocessing
Merupakan tahapan untuk membersihkan tweet dari Special Char, URL, Character yang tidak perlu dan mengubah Emoticon menjadi bahasa yang bisa dimengerti oleh manusia.
Fungsinya agar pengolahan tweet lebih mudah saat masuk metode utama.

In [28]:
#dependency
import tweepy
from textblob import TextBlob
import numpy as np
import csv
import datetime
import nltk
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
import re
import pandas as pd
from itertools import zip_longest

In [29]:
twToken = TweetTokenizer()
tweetStorage = []
brendaTwWithEmoji = []
emoji = []
kamus = {}
uname = 'res'

In [30]:
#Baca Data tweet
data = pd.read_csv('res.csv', usecols=[0, 1], names=['date', 'tweet'])
for line in data['tweet']:
    tweetStorage.append(line)

In [31]:
#baca library Emoticon
with open("Emoticon_Lib.csv", 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        emoji.append(row)

In [32]:
#load emoticon kedalam dictionary
for i in range(len(emoji)):
    a = emoji[i][0]
    b = emoji[i][1]
    kamus[a] = b

In [33]:
# fungsi untuk mengecek kunci sebuah dictionary
def checkKey(dict, key):  
    hasil = ''
    if key in dict:
        hasil = 'yes'
    else:
        hasil = 'No'
    return hasil

In [34]:
#fungsi untuk menconvert emoji kedalam bahasa manusia berdasarkan kamus emoji
def replace_emoji(key):
    kunci = 'U+'+key
    tes = checkKey(kamus, kunci)
    hasil = ''
    if tes == 'yes':
        hasil = '#'+kamus[kunci]+'#'
    elif tes == 'No':
        hasil = ''
    return hasil

# Bersihkan Tweet Dari Special Char, Convert Emoji 

In [35]:
for x in range(len(tweetStorage)):
    text = ''.join(tweetStorage[x])
    text = text[1::]
    text = text.replace("\\n\\", '')
    text = text.replace("'", ' ')
    text = text.replace("...", '')
    text = text.replace("\\", ' ')
    text = text.replace("(", '')
    text = text.replace(")", '')
    text = text.replace("w/", '')
    text = text.replace("U000", 'u')
    text = text.lower()
    sent = text.split()
    xMoji = []
    for t in range(len(sent)):
        u = sent[t]
        if u[0] == 'u':
            if len(u) > 1:
                if u[1] == '0' or u[1] == '1' or u[1] == '2' or u[1] == '3':
                    u = u.replace('u', '')
                    xMoji.append(u)
    for j in range(len(xMoji)):
        moji = xMoji[j].upper()
        ress = replace_emoji(moji)
        has = 'u'+xMoji[j]
        text = text.replace(has, ress)
        text = re.sub("\s\s+", " ", text)
    brendaTwWithEmoji.append(text)

In [36]:
bundle = [data['date'], brendaTwWithEmoji]
export_data = zip_longest(*bundle, fillvalue='')
with open('res1.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerows(export_data)
myfile.close()

In [37]:
data = pd.read_csv('res1.csv', usecols=[0, 1], names=['date', 'tweet'])
pd.set_option('display.max_colwidth', -1)
data

,date,tweet
0,8-Jul-20,#plnvangke @pln_123 cuma pake 350 kwh kok dibebankan 1.562 kwh? https://t.co/o8dyn3xoqd
1,8-Jul-20,asal usul tagihan listrik ajaib by @pln_123 #plnvangke https://t.co/j4pphxfaro
2,8-Jul-20,@pln_123 ini pln kalo nyatat kira2 dong jangan ngawur. asal jeplak aja karena banyak kegiatan dirumah. bisa diliat itu naiknya ga kira3#plnvangke https://t.co/qnqk0yvcx5
3,8-Jul-20,#plnvangke woii anjir akhir2 ini hampir setiap hari pasti mati lampu. lo kira gue gaada kerjaan?????!!!!! skripsi gue di ujung tanduk nih #pouting face# #pouting face# #pouting face#
4,8-Jul-20,"dear #plnvangke pak dengerin nih listrik mati dah 6 jam gw kebelet ee kamar mandi gelap ahirnya w paksain tuh gelap""an jongkok dah tu perasaan posisi udah pas nih d atas lobang ternyata kondisi pantat ketinggian al hasil kemana"" dah tu bubur coklat :3 nanjeng lah"
...,...,...
464,2-Jul-20,@indruxx20 @tiaputriput @alamauliya @dr_tompi ini makin ngaco @pln_123 ada byk yg komplain tagihan listrik naik ini mencurigakan. harus dibongkar ini. alasan kalian jgn2 semua sama pd pelanggan yg komplain
465,2-Jul-20,@indruxx20 @tiaputriput @alamauliya @dr_tompi kita komplain di twitter ke @pln_123 dan mrk bls d dm. coba teman2 blg alasan mrk knp tagihan listrik naik? jangan2 semua alasannya sama
466,2-Jul-20,@pln_123 tagihan listrik saya bulan ini naik 2x lipat sementara pemakaian seperti bulan-bulan sebelumnya. untuk komplain saya harus kemana ya? thanks.
467,2-Jul-20,@pln_123 parah banget sih ini akhirnya ngerasain juga tagihan listrik naik hampir 2x lipat di bulan ini. mau komplain kemana ya?


#  Bersihkan data Dari URL

In [38]:
userTw = []
newSentence = []

In [39]:
#baca data hasil pembersihan algoritma sebelumnya
data = pd.read_csv('res1.csv', usecols=[0, 1], names=['date', 'tweet'])
for line in data['tweet']:
    userTw.append(line)

In [40]:
#algoritma pembersihan URL
for i in range(len(userTw)):
    sent = ''.join(userTw[i])
    token = sent.split()
    newToken = sent.split()
    for x in range(len(token)):
        # tambahkan fitur yg mau dihapus
        if 'http' in token[x]:
            newToken.remove(token[x])
        elif 'ue6f4' in token[x]:
            newToken.remove(token[x])
    concate = ' '.join(newToken)
    newSentence.append(concate)

In [41]:
bundle = [data['date'], newSentence]
export_data = zip_longest(*bundle, fillvalue='')
with open('finalress.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerows(export_data)
myfile.close()

In [42]:
data = pd.read_csv('finalress.csv', usecols=[0, 1], names=['date', 'tweet']) #save as csv
pd.set_option('display.max_colwidth', -1)
data

,date,tweet
0,8-Jul-20,#plnvangke @pln_123 cuma pake 350 kwh kok dibebankan 1.562 kwh?
1,8-Jul-20,asal usul tagihan listrik ajaib by @pln_123 #plnvangke
2,8-Jul-20,@pln_123 ini pln kalo nyatat kira2 dong jangan ngawur. asal jeplak aja karena banyak kegiatan dirumah. bisa diliat itu naiknya ga kira3#plnvangke
3,8-Jul-20,#plnvangke woii anjir akhir2 ini hampir setiap hari pasti mati lampu. lo kira gue gaada kerjaan?????!!!!! skripsi gue di ujung tanduk nih #pouting face# #pouting face# #pouting face#
4,8-Jul-20,"dear #plnvangke pak dengerin nih listrik mati dah 6 jam gw kebelet ee kamar mandi gelap ahirnya w paksain tuh gelap""an jongkok dah tu perasaan posisi udah pas nih d atas lobang ternyata kondisi pantat ketinggian al hasil kemana"" dah tu bubur coklat :3 nanjeng lah"
...,...,...
464,2-Jul-20,@indruxx20 @tiaputriput @alamauliya @dr_tompi ini makin ngaco @pln_123 ada byk yg komplain tagihan listrik naik ini mencurigakan. harus dibongkar ini. alasan kalian jgn2 semua sama pd pelanggan yg komplain
465,2-Jul-20,@indruxx20 @tiaputriput @alamauliya @dr_tompi kita komplain di twitter ke @pln_123 dan mrk bls d dm. coba teman2 blg alasan mrk knp tagihan listrik naik? jangan2 semua alasannya sama
466,2-Jul-20,@pln_123 tagihan listrik saya bulan ini naik 2x lipat sementara pemakaian seperti bulan-bulan sebelumnya. untuk komplain saya harus kemana ya? thanks.
467,2-Jul-20,@pln_123 parah banget sih ini akhirnya ngerasain juga tagihan listrik naik hampir 2x lipat di bulan ini. mau komplain kemana ya?
